<a href="https://colab.research.google.com/github/chudriana/Fuzzy_Data_Matching/blob/main/fuzzy_matching_NN%2Bsentence_transformers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
payroll = '/content/drive/MyDrive/MAPPING MITRA ID - KSM/FUZZY MATCHING/payroll.csv'
mitra = '/content/drive/MyDrive/MAPPING MITRA ID - KSM/FUZZY MATCHING/mitra.csv'

In [ ]:
df_mitra = pd.read_csv(mitra)
df_payroll = pd.read_csv(payroll)

In [ ]:
def preprocess_name(name):
    if pd.isna(name):
        return ""
    name = str(name).lower()
    name = re.sub(r'\b(pt|cv)\b', '', name)
    name = re.sub(r'[^a-z0-9\s]', '', name)
    # name = re.sub(r'\s+', ' ', name).strip()
    return name

In [ ]:
df_payroll['mitra_name_clean'] = df_payroll['mitra'].apply(preprocess_name)
df_mitra['mitra_name_clean'] = df_mitra['mitra'].apply(preprocess_name)

In [ ]:
df_mitra['embedding'] = df_mitra['mitra_name_clean'].apply(lambda x: model.encode(x))
df_payroll['embedding'] = df_payroll['mitra_name_clean'].apply(lambda x: model.encode(x))


In [ ]:
def fuzzy_match_embedding(source_df, target_df, source_col='embedding', target_col='embedding', threshold=0.85):
    source_embeddings = np.vstack(source_df[source_col].values)
    target_embeddings = np.vstack(target_df[target_col].values)

    nn = NearestNeighbors(n_neighbors=1, metric='cosine')
    nn.fit(target_embeddings)
    distances, indices = nn.kneighbors(source_embeddings)

    similarity_scores = 1 - distances.flatten()

    match_results = []
    for i, score in enumerate(similarity_scores):
        if score >= threshold:
            matched_row = {
                'payroll_name': source_df.iloc[i]['mitra'],
                'payroll_name_clean': source_df.iloc[i]['mitra_name_clean'],
                'matched_name': target_df.iloc[indices[i][0]]['mitra'],
                'matched_name_clean': target_df.iloc[indices[i][0]]['mitra_name_clean'],
                'mitra_id': target_df.iloc[indices[i][0]]['mitra_id'],
                # 'parent_id': target_df.iloc[indices[i][0]]['parent_id'],
                'similarity': score
            }
            match_results.append(matched_row)

    return pd.DataFrame(match_results)


In [ ]:
matched_df = fuzzy_match_embedding(df_payroll, df_mitra, threshold=0.85)
matched_df.head()

In [ ]:
matched_df.to_csv('/content/drive/MyDrive/MAPPING MITRA ID - KSM/FUZZY MATCHING/matched_mitra_id_all-mpnet-base-v2.csv', index=False)